In [1]:
import cv2
import numpy as np

In [2]:
def face_catch(gray):
    pathf = '/Users/chenyuqing/opencv/data/haarcascades/haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(pathf)
    faceRects = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(50, 50))
    if len(faceRects):
        for faceRect in faceRects:
            x, y, w, h = faceRect
            cv2.rectangle(img, (x, y), (x + h, y + w), (0, 255, 0), 2)  # 框出人脸
    cv2.imshow("Image", img)

In [3]:
def moving_recognition(firstFrame, gray, frame):
    #计算不同
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)
    #填充孔洞
    thresh = cv2.dilate(thresh, None, iterations=3)
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) < 1000: #面积阈值
            continue
        #计算最小外接矩形
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    if contours:
        face_catch(gray)
    cv2.imshow('frame_with_result', frame)
    cv2.imshow('thresh', thresh)
    cv2.imshow('frameDiff', frameDelta)

    
    

In [4]:

cap = cv2.VideoCapture(0)
refresh = 50
firstFrame = None
if(cap.isOpened()):
    while(True):
        ret, frame = cap.read()
        #最后一帧退出
        if not ret:
            break;
        #调整大小
        frame = cv2.resize(frame, (640, 360), interpolation=cv2.INTER_CUBIC)
        #转换灰度图
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #背景刷新率
        if refresh == 50:
            refresh = 0
            firstFrame = gray
        moving_recognition(firstFrame, gray, frame)
        #处理按键效果
        key = cv2.waitKey(60) & 0xff
        if key == 27:
            break
        elif key == ord(' '):
            cv2.waitKey(0)
        
    cap.release()
cv2.destroyAllWindows()

TypeError: Expected Ptr<cv::UMat> for argument 'src'